In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import os
import shutil

from torchinfo import summary
import torch.utils.tensorboard as tb

import models
import mnist

torch.manual_seed(0);

In [3]:
# import tempfile
# tb_log_dir = tempfile.mkdtemp()
user = os.getlogin()
# tb_log_dir = os.path.join(cp.data_dir, 'tensorboard/', user)
tb_log_dir = f'/tmp/tensorboard/{user}'
# print(tb_log_dir)
if os.path.exists(tb_log_dir):
    shutil.rmtree(tb_log_dir)
    

In [4]:
logger = tb.SummaryWriter(tb_log_dir)

In [5]:

batch_idx, (X_batch, Y_batch) = next(enumerate(mnist.train_loader))
def calc_pheo_fitness(net):
    Y_batch_pred = net(X_batch)
    n_correct = (Y_batch_pred.argmax(dim=-1)==Y_batch).sum().item()
    loss = mnist.loss_func(Y_batch_pred.log(), Y_batch).item()
    accuracy = n_correct/len(Y_batch)
    return loss, accuracy

In [6]:
model = models.SmallNet

def to_np_array(population):
    pop = np.empty(len(population), dtype=object)
    for i in range(len(pop)):
        pop[i] = population[i]
    return pop

def get_init_population():
    pop = [nn.utils.parameters_to_vector(model().parameters()).detach() for _ in range(100)]
    pop = to_np_array(pop)
    return pop

def geno2pheno():
    pass
def pheno2geno():
    pass

def calc_fitnesses(population):
    agent = model()
    fitnesses = []
    fitnesses_acc = []
    for dna in population:
        nn.utils.vector_to_parameters(dna, agent.parameters())
#         fitness = -perform_stats(agent, train_loader, n_batches=1, show_stats=False, tqdm=None)[0]
        nll, acc = calc_pheo_fitness(agent)
        fitnesses.append(-nll)
        fitnesses_acc.append(acc)
    return fitnesses, fitnesses_acc

def calc_mutate(pop):
    mutant = [d+1e-2*torch.randn_like(d) for d in pop]
    return mutant

def calc_crossover(pop1, pop2):
    cross = [torch.stack([d1, d2], dim=0).mean(dim=0) for d1, d2 in zip(pop1, pop2)]
    return cross
    
def calc_next_population(population, fitnesses):
    N = len(population)
    npop = []
    
    n_keep_best = int(.1*N)
    npop.extend(population[np.argsort(fitnesses)[-n_keep_best:]])
    
    prob = torch.tensor(fitnesses).softmax(dim=-1).numpy()
    
    
    n_parents = int(.3*N)
    n_mutants = N-n_keep_best-n_parents
    
    mutants = np.random.choice(population, size=n_mutants, p=prob)
    mutants = calc_mutate(mutants)
    parents1 = np.random.choice(population, size=n_parents, p=prob)
    parents2 = np.random.choice(population, size=n_parents, p=prob)
    children = calc_crossover(parents1, parents2)
    
    npop.extend(mutants)
    npop.extend(children)
    
    npop = to_np_array(npop)
    return npop
    
    

population = get_init_population()
for gen_idx in tqdm(range(1000)):
    fitnesses, fitnesses_acc = calc_fitnesses(population)
    population = calc_next_population(population, fitnesses)
    
    logger.add_histogram('fitness', np.array(fitnesses), global_step=gen_idx)
    logger.add_histogram('accuracies', np.array(fitnesses_acc), global_step=gen_idx)


  0%|          | 0/1000 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/akarshkumar0101/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-29c71782c65a>", line 68, in <module>
    fitnesses, fitnesses_acc = calc_fitnesses(population)
  File "<ipython-input-6-29c71782c65a>", line 26, in calc_fitnesses
    nll, acc = calc_pheo_fitness(agent)
  File "<ipython-input-5-458c568e3514>", line 3, in calc_pheo_fitness
    Y_batch_pred = net(X_batch)
  File "/Library/Python/3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/Users/akarshkumar0101/evolved-neuroevolution/models.py", line 30, in forward
    x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
  File "/Library/Python/3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/Library/Python/3.8/s


KeyboardInterrupt



# Now, we are going to learn the crossover function

In [7]:
net = model()
v = torch.nn.utils.parameters_to_vector(net.parameters())
torch.nn.utils.vector_to_parameters(v, net.parameters())
dna_len = len(v)

In [8]:
class CONet(nn.Module):
    def __init__(self):
        super().__init__()
        n = dna_len
#         self.lin1 = nn.Linear(n, n//2)
#         self.lin2 = nn.Linear(n//2, n//4)
#         self.lin3 = nn.Linear(64, n//2)
#         self.lin4 = nn.Linear(n//2, n)

        self.lin1 = nn.Linear(n, n)
        self.lin2 = nn.Linear(2*n, n)
        self.lin3 = nn.Linear(n, n)

    def forward(self, x):
        bs = len(x)
        x = torch.sigmoid(self.lin1(x))
        x = x.reshape(bs, -1)
        x = torch.sigmoid(self.lin2(x))
        x = self.lin3(x)
        
#         x = torch.sigmoid(self.lin1(x))
#         x = torch.sigmoid(self.lin2(x))
#         x = x.reshape(len(x), -1)
#         x = torch.sigmoid(self.lin3(x))
#         x = self.lin4(x)
        return x

conet = CONet()
summary(conet, input_size=(1, 2, dna_len))

Layer (type:depth-idx)                   Output Shape              Param #
CONet                                    --                        --
├─Linear: 1-1                            [1, 2, 130]               17,030
├─Linear: 1-2                            [1, 130]                  33,930
├─Linear: 1-3                            [1, 130]                  17,030
Total params: 67,990
Trainable params: 67,990
Non-trainable params: 0
Total mult-adds (M): 0.07
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.27
Estimated Total Size (MB): 0.28

# two ideas need to pursued:
- having a population of agents and a separate population of crossover networks
- having a population of male and female agents, where female agents consist of an overhead crossover network

## below is implementation of a male-female system

In [11]:
class DNA():
    def __init__(self):
        self.geno = None
        self.co_geno = None # will be None if male


model = models.SmallNet
comodel = CONet

def to_np_array(population):
    pop = np.empty(len(population), dtype=object)
    for i in range(len(pop)):
        pop[i] = population[i]
    return pop

def get_init_population():
    pop = [DNA() for _ in range(100)]
    for i in range(100):
        pop[i].geno = nn.utils.parameters_to_vector(model().parameters()).detach()
    for i in range(50):
        pop[i].co_geno = nn.utils.parameters_to_vector(comodel().parameters()).detach()
    pop = to_np_array(pop)
    return pop[:50], pop[50:]

def geno2pheno():
    pass
def pheno2geno():
    pass

def calc_fitnesses(population):
    agent = model()
    fitnesses = []
    fitnesses_acc = []
    for dna in population:
        nn.utils.vector_to_parameters(dna.geno, agent.parameters())
#         fitness = -perform_stats(agent, train_loader, n_batches=1, show_stats=False, tqdm=None)[0]
        nll, acc = calc_pheo_fitness(agent)
        fitnesses.append(-nll)
        fitnesses_acc.append(acc)
    return fitnesses, fitnesses_acc

def calc_mutate(pop):
    mutant = [DNA() for _ in pop]
    for o, m in zip(pop, mutant):
        if o.geno is not None:
            m.geno = o.geno + 1e-2*torch.randn_like(o.geno)
        if o.co_geno is not None:
            m.co_geno = o.co_geno + 1e-2*torch.randn_like(o.co_geno)
    return mutant

def calc_crossover(popf, popm):
    children = [DNA() for _ in popf]
    conet = comodel()
    for i in range(len(children)):
        x = torch.stack([popf[i].geno, popm[i].geno], dim=0)
        cogeno = popf[i].co_geno
        nn.utils.vector_to_parameters(cogeno, conet.parameters())
        y = conet(x[None])[0]
        children[i].geno = y
    for i in torch.randperm(len(popf))[:len(popf)//2]:
        children[i].co_geno = popf[i].co_geno
        
    childrenf = [c for c in children if c.co_geno is not None]
    childrenm = [c for c in children if c.co_geno is None]
    return childrenf, childrenm
    
def calc_next_population(pop_f, pop_m):
    fit_f, fit_acc_f = calc_fitnesses(pop_f)
    fit_m, fit_acc_m = calc_fitnesses(pop_m)
    
    N_f = len(pop_f)
    N_m = len(pop_m)
    
    npop = []
    
    n_keep_best = 5
    npop.extend(pop_f[np.argsort(fit_f)[-n_keep_best:]])
    npop.extend(pop_m[np.argsort(fit_m)[-n_keep_best:]])
    
    prob_f = torch.tensor(fit_f).softmax(dim=-1).numpy()
    prob_m = torch.tensor(fit_m).softmax(dim=-1).numpy()
    
#     n_parents = 30
#     n_mutants = 30
    n_parents = 10
    n_mutants = 40
    
    mutants_f = np.random.choice(pop_f, size=n_mutants, p=prob_f)
    mutants_m = np.random.choice(pop_m, size=n_mutants, p=prob_m)
    mutants_f = calc_mutate(mutants_f)
    mutants_m = calc_mutate(mutants_m)
    
    parents_f = np.random.choice(pop_f, size=n_parents, p=prob_f)
    parents_m = np.random.choice(pop_m, size=n_parents, p=prob_m)
    children_f, children_m = calc_crossover(parents_f, parents_m)
    
    npop.extend(mutants_f)
    npop.extend(mutants_m)
    npop.extend(children_f)
    npop.extend(children_m)
    
    npop_f = [c for c in npop if c.co_geno is not None]
    npop_m = [c for c in npop if c.co_geno is None]

    npop_f = to_np_array(npop_f)
    npop_m = to_np_array(npop_m)
    return npop_f, npop_m, fit_f, fit_m
    
    

pop_f, pop_m = get_init_population()
for gen_idx in tqdm(range(10000)):
    pop_f, pop_m, fit_f, fit_m = calc_next_population(pop_f, pop_m)
    
    logger.add_histogram('co female fitness histogram', np.array(fit_f), global_step=gen_idx)
    logger.add_histogram('co male fitness histogram', np.array(fit_m), global_step=gen_idx)
#     logger.add_histogram('accuracies histogram', np.array(fitnesses_acc), global_step=gen_idx)

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
conet = nn.Sequential(
    nn.Linear(len(v)*2, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, len(v)*2),
)
munet = nn.Sequential(
    nn.Linear(len(v), 100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 100),
    nn.ReLU(),
    nn.Linear(100, len(v)),
    nn.Tanh(),
)
summary(munet)

Layer (type:depth-idx)                   Param #
Sequential                               --
├─Linear: 1-1                            68,600
├─ReLU: 1-2                              --
├─Linear: 1-3                            5,050
├─ReLU: 1-4                              --
├─Linear: 1-5                            5,100
├─ReLU: 1-6                              --
├─Linear: 1-7                            69,185
├─Tanh: 1-8                              --
Total params: 147,935
Trainable params: 147,935
Non-trainable params: 0

In [48]:
torch.manual_seed(0)
rand_idx1 = torch.arange(1000)
rand_idx2 = torch.arange(1000)
rand_idx3 = torch.arange(1000)
class MuNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(50, 50)
        self.lin2 = nn.Linear(50, 50)
        self.lin3 = nn.Linear(50, 50)
        self.lin4 = nn.Linear(50, 50)

    def forward(self, x):
        x = torch.cat([x, torch.zeros(len(x), 1000-len(v))], dim=-1)
        x = x.reshape(-1, 20, 50)
        x = torch.relu(self.lin1(x))
        x = x.reshape(-1, 1000)[:, rand_idx1]
        x = x.reshape(-1, 20, 50)
        x = torch.relu(self.lin2(x))
        x = x.reshape(-1, 1000)[:, rand_idx2]
        x = x.reshape(-1, 20, 50)
        x = torch.relu(self.lin3(x))
        x = x.reshape(-1, 1000)[:, rand_idx3]
        x = x.reshape(-1, 20, 50)
        x = torch.relu(self.lin4(x))
        x = x.reshape(-1, 1000)[:, :len(v)]
        return x
    
class CONet(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(50, 50)
        self.lin2 = nn.Linear(50, 50)
        self.lin3 = nn.Linear(50, 50)
        self.lin4 = nn.Linear(50, 50)

    def forward(self, x):
        x = torch.cat([x, torch.zeros(len(x), 1000-len(v))], dim=-1)
        x = x.reshape(-1, 20, 50)
        x = torch.relu(self.lin1(x))
        x = x.reshape(-1, 1000)[:, rand_idx1]
        x = x.reshape(-1, 20, 50)
        x = torch.relu(self.lin2(x))
        x = x.reshape(-1, 1000)[:, rand_idx2]
        x = x.reshape(-1, 20, 50)
        x = torch.relu(self.lin3(x))
        x = x.reshape(-1, 1000)[:, rand_idx3]
        x = x.reshape(-1, 20, 50)
        x = torch.relu(self.lin4(x))
        x = x.reshape(-1, 1000)[:, :len(v)]
        return x
    
munet = MuNet()
print(munet(v[None]).shape)
summary(munet)

torch.Size([1, 685])


Layer (type:depth-idx)                   Param #
MuNet                                    --
├─Linear: 1-1                            2,550
├─Linear: 1-2                            2,550
├─Linear: 1-3                            2,550
├─Linear: 1-4                            2,550
Total params: 10,200
Trainable params: 10,200
Non-trainable params: 0

In [7]:
685*685

469225